In [ ]:
import numpy as np
import pandas as pd
from src.splitmeasfilter import SplitMeasFilter, dictToVec, vecToDict

# --- For this test to be self-contained, I will assume the class is available 
# --- or you paste the class definition above this block.

def generate_synthetic_data(n_qubits, shots, error_rate, ground_truth_dist):
    """
    Simulates noise by flipping bits with probability 'error_rate'.
    """
    noisy_counts = {}
    
    # Iterate over ground truth
    for state, count in ground_truth_dist.items():
        # Convert state string to list of bits
        bits = [int(b) for b in state]
        
        # For every shot of this state, apply noise
        for _ in range(count):
            noisy_bits = []
            for b in bits:
                # Flip with probability 'error_rate'
                if np.random.rand() < error_rate:
                    noisy_bits.append(1 - b)
                else:
                    noisy_bits.append(b)
            
            # Join back to string
            noisy_str = "".join(map(str, noisy_bits))
            noisy_counts[noisy_str] = noisy_counts.get(noisy_str, 0) + 1
            
    return noisy_counts

def run_test():
    print("=== Starting SplitMeasFilter DeNoise Unit Test ===\n")

    # Parameters
    QUBITS = [0, 1]
    SHOTS = 10000
    ERROR_RATE = 0.10  # 10% readout error on both 0 and 1
    SUCCESS_RATE = 1.0 - ERROR_RATE
    
    # A Bell State |+>: 50% '00' and 50% '11'
    ground_truth = {
        '00': int(0.5 * SHOTS),
        '11': int(0.5 * SHOTS)
    }
    print(f"1. Ground Truth (Target): {ground_truth}")

    # Noisy Data (Simulating the QPU)
    np.random.seed(42) # Fix seed for reproducibility
    noisy_data = generate_synthetic_data(len(QUBITS), SHOTS, ERROR_RATE, ground_truth)
    print(f"2. Noisy Data (Observed): {dict(sorted(noisy_data.items()))}")
    print(f"   (Note the appearance of '01' and '10' ghost states due to error)\n")

    # Initialize Filter 
    # We pass data=[] and device=None to avoid file loading/AWS calls
    rem = SplitMeasFilter(QUBITS, home_dir=None, data=[], device=None)
    
    # Instead of running inference(), we manually inject the known success rates.
    # We simulate that the filter learned the exact error rates from calibration.
    print("3. Injecting Calibration Data...")
    for q in QUBITS:
        # Create an array of "samples" that all equal the success rate
        # This mocks the posterior distribution having a mean of 0.90
        dummy_posterior = np.full(1000, SUCCESS_RATE)
        
        rem.post_marginals[f'Qubit{q}']['0'] = dummy_posterior
        rem.post_marginals[f'Qubit{q}']['1'] = dummy_posterior
        
    # Important: Force matrix creation after injection
    rem.create_filter_mat() 
    print("   Calibration Injected. Filter Matrices created.\n")

    # Run DeNoise (Standard Analytic)
    print("4. Running eff_DeNoise (Standard)...")
    cleaned_standard = rem.eff_DeNoise(noisy_data, method='mean', percentage=100, verbose=False)
    
    # Run DeNoise (Gradient Descent)
    print("5. Running eff_DeNoise (Gradient Descent)...")
    cleaned_gd = rem.eff_DeNoise(noisy_data, method='mean', percentage=100, GD=True, verbose=False)

    # Comparison and Scoring
    print("\n=== RESULTS COMPARISON ===")
    print(f"{'State':<6} | {'Truth':<10} | {'Noisy':<10} | {'Standard':<10} | {'GD Refined':<10}")
    print("-" * 60)
    
    all_keys = sorted(list(set(ground_truth.keys()) | set(noisy_data.keys())))

    for k in all_keys:
        truth = ground_truth.get(k, 0)
        noisy = noisy_data.get(k, 0)
        std_val = int(cleaned_standard.get(k, 0))
        gd_val = int(cleaned_gd.get(k, 0))        
        print(f"{k:<6} | {truth:<10} | {noisy:<10} | {std_val:<10} | {gd_val:<10}")

    print("\nSUCCESS: Denoising significantly reduced error.")

if __name__ == "__main__":
    # Ensure the SplitMeasFilter class is defined in this scope before running
    try:
        run_test()
    except NameError:
        print("Error: SplitMeasFilter class is not defined. Please run the class definition cell first.")

=== Starting SplitMeasFilter DeNoise Unit Test ===

1. Ground Truth (Target): {'00': 5000, '11': 5000}
2. Noisy Data (Observed): {'00': 4069, '01': 930, '10': 900, '11': 4101}
   (Note the appearance of '01' and '10' ghost states due to error)

3. Injecting Calibration Data...
   Calibration Injected. Filter Matrices created.

4. Running eff_DeNoise (Standard)...
5. Running eff_DeNoise (Gradient Descent)...
----  GD Step 1/50 complete...  ----
----  GD Step 2/50 complete...  ----
----  GD Step 3/50 complete...  ----
----  GD Step 4/50 complete...  ----
----  GD Step 5/50 complete...  ----
----  GD Step 6/50 complete...  ----
----  GD Step 7/50 complete...  ----
----  GD Step 8/50 complete...  ----
----  GD Step 9/50 complete...  ----
----  GD Step 10/50 complete...  ----
----  GD Step 11/50 complete...  ----
----  GD Step 12/50 complete...  ----
----  GD Step 13/50 complete...  ----
----  GD Step 14/50 complete...  ----
----  GD Step 15/50 complete...  ----
----  GD Step 16/50 complete

<Figure size 640x480 with 0 Axes>